In [60]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm, trange
from itertools import islice

tf.__version__

'2.2.0'

In [56]:
train = tf.data.experimental.make_csv_dataset("data/train.tsv",
                                           field_delim="\t",
                                           batch_size=10_000,  # increase for real work
#                                            compression_type="GZIP",
                                           label_name="Click",
                                           num_epochs=1)

test = tf.data.experimental.make_csv_dataset("data/test.tsv",
                                           field_delim="\t",
                                           batch_size=5,  # increase for real work
                                           label_name="Click",
                                           num_epochs=1)


In [57]:
x = next(iter(train))

#### Data normalization
Calculate mean and average of a column

In [58]:
def calculate_mean_and_sd(col):
    cnt, mean_sum, sd_sum = 0, 0, 0
    n_iter = 100
    for x, y in tqdm(islice(train, n_iter), total=n_iter):
        cnt += 1
        mean_sum += tf.math.reduce_mean(tf.cast(x[col], tf.float32))
        sd_sum += tf.math.reduce_std(tf.cast(x[col], tf.float32))
    mean = mean_sum / cnt
    sd = sd_sum / cnt
    return mean.numpy(), sd.numpy()

In [59]:
numeric_columns = ["Depth", "Position", "Gender", "Age", "UserID", "AdvertiserId"]
means, sds = [], []
for col in numeric_columns:
    mean, sd = calculate_mean_and_sd(col)
    means.append(mean)
    sds.append(sd)

In [65]:
stats = pd.DataFrame({"column": numeric_columns, "mean": means, "sd": sds})
stats.to_csv("data/stats.csv", index=False)
pd.read_csv("data/stats.csv")

,column,mean,sd
0,Depth,1.898353e+00,6.924177e-01
1,Position,1.454778e+00,6.157744e-01
2,Gender,1.441896e+00,5.291840e-01
3,Age,3.218050e+00,1.472307e+00
4,UserID,4.482932e+06,4.967242e+06
5,AdvertiserId,2.292905e+04,1.135397e+04
